In [1]:
# ================================================
# ✅ 1️⃣ LIBRARIES & SETUP
# ================================================
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from torchvision import transforms
import numpy as np

# ================================================
# ✅ 2️⃣ PATHS
# ================================================
image_dir = "/kaggle/input/basem/images"
input_csv = "/kaggle/input/basem/dataset.csv"

# ================================================
# ✅ 3️⃣ LOAD & PREPROCESS CSV
# ================================================
df = pd.read_csv(input_csv)

existing_data = []
for _, row in df.iterrows():
    image_filename = row['image_path']
    full_image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(full_image_path):
        label_converted = row['label 2'] - 1
        existing_data.append({
            'Image_path': full_image_path,
            'Label_Sentiment': label_converted
        })

processed_df = pd.DataFrame(existing_data)

# ================================================
# ✅ 4️⃣ DATA SPLITTING
# ================================================
train_df, temp_df = train_test_split(processed_df, test_size=0.3, stratify=processed_df['Label_Sentiment'], random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_Sentiment'], random_state=42)

# Save splits for reference
for df_name, df_ in [('train', train_df), ('test', test_df), ('val', val_df)]:
    df_['label'] = df_['Label_Sentiment']
    df_.to_csv(f'/kaggle/working/{df_name}_vision_only.csv', index=False)

# ================================================
# ✅ 5️⃣ DEVICE SETUP
# ================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ================================================
# ✅ 6️⃣ IMAGE TRANSFORMS
# ================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================================
# ✅ 7️⃣ VISION DATASET
# ================================================
class VisionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image_path']).convert('RGB')
        label = row['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# ================================================
# ✅ 8️⃣ DATALOADERS
# ================================================
batch_size = 16

train_dataset = VisionDataset(train_df, transform=train_transform)
val_dataset = VisionDataset(val_df, transform=val_test_transform)
test_dataset = VisionDataset(test_df, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ================================================
# ✅ 9️⃣ CNN MODEL DEFINITION
# ================================================
class VisionCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(VisionCNN, self).__init__()
        
        # First Convolutional Block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)
        
        # Second Convolutional Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.25)
        
        # Third Convolutional Block
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(0.25)
        
        # Fourth Convolutional Block
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(512)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.dropout4 = nn.Dropout2d(0.25)
        
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(512, 256)
        self.fc_bn1 = nn.BatchNorm1d(256)
        self.fc_dropout1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(256, 128)
        self.fc_bn2 = nn.BatchNorm1d(128)
        self.fc_dropout2 = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # First block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # Second block
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Third block
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        # Fourth block
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool4(x)
        x = self.dropout4(x)
        
        # Global average pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc_bn1(self.fc1(x)))
        x = self.fc_dropout1(x)
        
        x = F.relu(self.fc_bn2(self.fc2(x)))
        x = self.fc_dropout2(x)
        
        x = self.classifier(x)
        
        return x

# ================================================
# ✅ 🔟 MODEL INITIALIZATION
# ================================================
model = VisionCNN(num_classes=3).to(device)

# Print model summary
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {count_parameters(model):,}")

# ================================================
# ✅ 1️⃣1️⃣ LOSS & OPTIMIZER
# ================================================
# Calculate class weights for balanced training
class_weights = train_df['label'].value_counts().sort_index().tolist()
total = sum(class_weights)
weights = [total / c for c in class_weights]
print(f"Class distribution: {class_weights}")
print(f"Class weights: {weights}")

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

# ================================================
# ✅ 1️⃣2️⃣ TRAINING LOOP
# ================================================
num_epochs = 20
patience = 3
patience_counter = 0
best_val_loss = float('inf')

training_history = {
    'train_loss': [],
    'val_loss': [],
    'val_accuracy': []
}

for epoch in range(num_epochs):
    # ============================================================
    # TRAINING PHASE
    # ============================================================
    model.train()
    total_train_loss = 0
    train_predictions = []
    train_labels = []

    for images, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        
        # Store predictions for training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_predictions.extend(predicted.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = accuracy_score(train_labels, train_predictions)

    # ============================================================
    # VALIDATION PHASE
    # ============================================================
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            
            # Store predictions for metrics
            _, predicted = torch.max(outputs.data, 1)
            val_predictions.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_predictions)
    
    # Update learning rate scheduler
    scheduler.step(avg_val_loss)
    
    # Store history
    training_history['train_loss'].append(avg_train_loss)
    training_history['val_loss'].append(avg_val_loss)
    training_history['val_accuracy'].append(val_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 50)

    # ============================================================
    # EARLY STOPPING CHECK
    # ============================================================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_vision_model.pt")
        print("✅ Validation loss improved — model saved.")
    else:
        patience_counter += 1
        print(f"⏰ No improvement — patience {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

# ================================================
# ✅ 1️⃣3️⃣ FINAL TEST EVALUATION
# ================================================
print("\n🔍 Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_vision_model.pt"))
model.eval()

test_predictions = []
test_labels = []
total_test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Test Evaluation"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate final metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='weighted')
cm = confusion_matrix(test_labels, test_predictions)

print("\n" + "="*60)
print("📊 FINAL TEST RESULTS (VISION-ONLY CNN):")
print("="*60)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Loss: {total_test_loss/len(test_loader):.4f}")
print(f"\nConfusion Matrix:\n{cm}")

# ================================================
# ✅ 1️⃣4️⃣ DETAILED CLASSIFICATION REPORT
# ================================================
from sklearn.metrics import classification_report

print("\n📋 DETAILED CLASSIFICATION REPORT:")
print("="*60)
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(test_labels, test_predictions, target_names=target_names))

# ================================================
# ✅ 1️⃣5️⃣ TRAINING HISTORY SUMMARY
# ================================================
print("\n📈 TRAINING HISTORY SUMMARY:")
print("="*60)
print(f"Best Validation Loss: {min(training_history['val_loss']):.4f}")
print(f"Best Validation Accuracy: {max(training_history['val_accuracy']):.4f}")
print(f"Final Training Loss: {training_history['train_loss'][-1]:.4f}")
print(f"Final Validation Loss: {training_history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {training_history['val_accuracy'][-1]:.4f}")

# Save training history
import pickle
with open('/kaggle/working/vision_training_history.pkl', 'wb') as f:
    pickle.dump(training_history, f)

print("\n✅ Vision-only CNN training completed successfully!")
print("📁 Model saved as 'best_vision_model.pt'")
print("📁 Training history saved as 'vision_training_history.pkl'")

Using device: cuda
Total trainable parameters: 4,854,595
Class distribution: [1404, 1237, 515]
Class weights: [2.247863247863248, 2.551333872271625, 6.128155339805825]


Validation Epoch 1: 100%|██████████| 29/29 [00:15<00:00,  1.85it/s]


Epoch [1/20]
Train Loss: 1.1560 | Train Acc: 0.3767
Val Loss: 1.0546 | Val Acc: 0.4213
Learning Rate: 0.000100
--------------------------------------------------
✅ Validation loss improved — model saved.


Validation Epoch 2: 100%|██████████| 29/29 [00:11<00:00,  2.43it/s]


Epoch [2/20]
Train Loss: 1.1456 | Train Acc: 0.3894
Val Loss: 1.0285 | Val Acc: 0.4723
Learning Rate: 0.000100
--------------------------------------------------
✅ Validation loss improved — model saved.


...

Epoch [12/20]
Train Loss: 1.0419 | Train Acc: 0.4908
Val Loss: 0.9672 | Val Acc: 0.5211
Learning Rate: 0.000100
--------------------------------------------------
⏰ No improvement — patience 2/3


Validation Epoch 13: 100%|██████████| 29/29 [00:11<00:00,  2.42it/s]


Epoch [13/20]
Train Loss: 1.0432 | Train Acc: 0.4810
Val Loss: 0.9704 | Val Acc: 0.5188
Learning Rate: 0.000050
--------------------------------------------------
⏰ No improvement — patience 3/3
🛑 Early stopping triggered at epoch 13

🔍 Loading best model for final evaluation...


Final Test Evaluation: 100%|██████████| 57/57 [00:31<00:00,  1.79it/s]


📊 FINAL TEST RESULTS (VISION-ONLY CNN):
Test Accuracy: 0.5687
Test Precision: 0.5766
Test Recall: 0.5687
Test F1-Score: 0.5695
Test Loss: 0.9778

Confusion Matrix:
[[230 138  34]
 [ 90 220  43]
 [ 26  58  63]]

📋 DETAILED CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    Negative       0.66      0.57      0.61       402
     Neutral       0.53      0.62      0.57       353
    Positive       0.45      0.43      0.44       147

    accuracy                           0.57       902
   macro avg       0.55      0.54      0.54       902
weighted avg       0.58      0.57      0.57       902


📈 TRAINING HISTORY SUMMARY:
Best Validation Loss: 0.9652
Best Validation Accuracy: 0.5698
Final Training Loss: 1.0432
Final Validation Loss: 0.9704
Final Validation Accuracy: 0.5188

✅ Vision-only CNN training completed successfully!
📁 Model saved as 'best_vision_model.pt'
📁 Training history saved as 'vision_training_history.pkl'
